# Regressão com Máquinas de Vetores de Suporte
SVR

In [45]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
py.offline.init_notebook_mode(connected = True)

#### Carrega os índices do ibc-br, dólar e igp-m mensais de Jan/2003 a Dez/2022

In [69]:
# gerando os dados
df = pd.read_csv('ds_indices.csv', sep = ';', decimal=',', thousands = '.', date_parser = 'data')
df.head()

,data,igp_m,dolar,ibc_br
0,01/01/03,277.173,3.4384,96.15
1,01/02/03,283.506,3.5908,98.67
2,01/03/03,287.855,3.4469,103.41
3,01/04/03,290.512,3.1187,102.19
4,01/05/03,289.747,2.9557,100.30


In [145]:
fig = px.scatter_3d(df, x="igp_m", y="dolar", z="ibc_br")
fig.show()

#### A regressão será calcular o ibc_br em função do dólar e do igp_m

In [72]:
X = df.drop(['ibc_br', 'data'], axis = 1)
Y = df['ibc_br']

In [73]:
x = X.to_numpy()
y = Y.to_numpy()

In [74]:
# criação do modelo com a SVR
svr = SVR(C=1.0, epsilon=0.2, kernel='linear')
svr.fit(x, y) 

SVR(epsilon=0.2, kernel='linear')

In [77]:
# gera previsões para outros valores
y_pred = svr.predict(x)

In [147]:
# avalia o erro das previsões
print('MSE: %.2f' % mean_squared_error(y, y_pred))
print('RMSE: %.2f' % np.sqrt(mean_squared_error(y, y_pred)))
print('R2: %.2f'% r2_score(y, y_pred))

MSE: 58.39
RMSE: 7.64
R2: 0.69


#### o primeiro coeficiente é o do igp_m e o segundo é o do dólar

In [95]:
coefs = svr.coef_
coefs

array([[  0.097337  , -13.37013092]])

In [96]:
intercept = svr.intercept_
intercept

array([116.57900013])

In [134]:
#calcula o ibc_br utilizando os coeficientes de o igp_m e do dólar e o intercepto
ibc_br_hat=[(intercept+coefs[0,0]*igp_m+coefs[0,1]*dolar).ravel()[0] for igp_m, dolar in zip(x[:,0], x[:,1])]

In [146]:
layout = go.Layout(title = "Previsão do IBC-Br em função do Dólar e do IGP-M",
                  scene = dict(
                      xaxis = dict(title = 'igp-m'),
                      yaxis = dict(title = 'dólar'),
                      zaxis = dict(title = 'ibc-br')))
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter3d(x=df.igp_m, y=df.dolar, z=df.ibc_br, name = 'real'))
fig.add_trace(go.Scatter3d(x=df.igp_m, y=df.dolar, z=ibc_br_hat, name = 'predito'))
fig.show()